In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

#from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn import metrics

In [3]:
df = pd.read_csv('df_final.csv', sep='\t')
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[0], axis=1)
df.head()

,arquivo,nome,ano,km,cor,cambio,portas,preco
0,./dados/dados_carro\Carro-100.html,Chevrolet Onix 1.0 LT (Flex) 2020,2020 / 2020,0.000,Laranja,manual,3.0,"59.890,00"
1,./dados/dados_carro\Carro-1004.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2019 / 2019,41.070,Branco,manual,2.0,"45.900,00"
2,./dados/dados_carro\Carro-1005.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,48.654,Prata,manual,4.0,"45.900,00"
3,./dados/dados_carro\Carro-1008.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,19.000,Prata,manual,2.0,"45.990,00"
4,./dados/dados_carro\Carro-1009.html,Fiat Strada Hard Working 1.4 (Flex) (Cabine Si...,2018 / 2019,24.715,Branco,manual,2.0,"45.990,00"


In [4]:
def convert_prices(df): 
    for x in range(100):
        if x >= 10:
            str_replace = ',' + str(x)
        else:    
            str_replace = ',0' + str(x)
        df['preco'] = df.preco.replace({str_replace:''}, regex=True)
    
    for i in range(len(df)) : 
        df['preco'][i] = df['preco'][i].replace('.', '')
    
    df['preco'] = df['preco'].astype(float)
    
    return df

In [5]:
df = convert_prices(df)

<ipython-input-4-f4cdf1d93ebb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preco'][i] = df['preco'][i].replace('.', '')


In [6]:
type(df['preco'][1])

numpy.float64

In [7]:
df['preco']

0       59890.0
1       45900.0
2       45900.0
3       45990.0
4       45990.0
         ...   
1706    45590.0
1707    45730.0
1708    45890.0
1709    45890.0
1710    45890.0
Name: preco, Length: 1711, dtype: float64

### Criando a baseline

In [8]:
df.describe()

,km,portas,preco
count,1711.000000,1486.000000,1711.000000
mean,30.362317,3.983176,52137.331970
std,75.420014,0.837898,18675.162781
min,0.000000,2.000000,15900.000000
25%,0.000000,4.000000,39900.000000
50%,13.000000,4.000000,46500.000000
75%,40.692000,4.000000,60714.500000
max,999.999000,5.000000,148000.000000


In [9]:
y = df.preco

In [10]:
y.head(3)

0    59890.0
1    45900.0
2    45900.0
Name: preco, dtype: float64

In [11]:
cars_features = ['km', 'portas']
X = df[cars_features]

In [12]:
X.head(3)

,km,portas
0,0.000,3.0
1,41.070,2.0
2,48.654,4.0


In [14]:
X.fillna(X.mean(), inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [15]:
#train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### RandomForestRegressor

In [22]:
rf = RandomForestRegressor(n_estimators=20, random_state=0)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [23]:
print('RF - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_pred))
print('RF - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, rf_pred))
print('RF - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_pred)))

RF - MAE - Mean Absolute Error: 12130.13714621856
RF - MSE - Mean Squared Error: 289786666.3864076
RF - RMSE - Root Mean Squared Error: 17023.121522987716


#### LogisticRegression

In [24]:
lr = LogisticRegression(solver='liblinear', multi_class='ovr')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

In [25]:
print('LR - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, lr_pred))
print('LR - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, lr_pred))
print('LR - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, lr_pred)))

LR - MAE - Mean Absolute Error: 13086.900874635568
LR - MSE - Mean Squared Error: 384705427.8746356
LR - RMSE - Root Mean Squared Error: 19613.90904115331


#### DecisionTreeRegressor

In [26]:
dt = DecisionTreeRegressor(max_leaf_nodes=10, random_state=1)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

In [31]:
print('DT - MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, dt_pred))
print('DT - MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, dt_pred))
print('DT - RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, dt_pred)))

DT - MAE - Mean Absolute Error: 11862.036106689564
DT - MSE - Mean Squared Error: 276756377.7213516
DT - RMSE - Root Mean Squared Error: 16635.996445099154
